In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import pandas as pd

from tvb_inversion.base.diagnostics import (zscore, shrinkage)
from tvb_inversion.sbi.plot import (
    plot_posterior_samples_model_parameters,
    plot_posterior_samples_global_parameters
)

%load_ext autoreload
%autoreload 2

In [ ]:
# run_id = "2023-03-10_1849"  # around ground truth
# run_id = "2023-03-14_2001"  # shifted from ground truth
# run_id = "2023-03-15_1303"  # around ground truth
run_id = "2023-03-15_1436"  # shifted from ground truth

posterior_samples = np.load(f"sbi_data/posterior_samples_{run_id}.npy")
prior_samples = np.load(f"sbi_data/prior_samples_{run_id}.npy")
test_samples = np.load(f"sbi_data/test_samples_{run_id}.npy")
X = np.load(f"sbi_data/simulation_{run_id}.npy")
training_sims = np.load(f"sbi_data/training_sims_{run_id}.npy")
test_sims = np.load(f"sbi_data/test_sims_{run_id}.npy")
with open(f"sbi_data/sim_params_{run_id}.json", "r") as f:
    simulation_params = json.load(f)
with open(f"sbi_data/inference_params_{run_id}.json", "r") as f:
    inference_params = json.load(f)
with open(f"sbi_data/summary_{run_id}.json", "r") as f:
    summary = json.load(f)

In [ ]:
training_sims.shape, test_sims.shape, prior_samples.shape, test_samples.shape

In [ ]:
inference_params

In [ ]:
f1 = plt.figure(figsize=(12, 8))
plt.plot(summary["training_log_probs"], label="training")
plt.plot(summary["validation_log_probs"], label="validation")
plt.xlabel("epochs", size=16)
plt.ylabel("log probability", size=16)
plt.tick_params(axis="both", labelsize=16)
plt.legend(fontsize=16)
plt.show()

In [ ]:
df = pd.DataFrame(
    data=posterior_samples[:, :, :len(inference_params["model_a"])].reshape(-1, len(inference_params["model_a"])),
    columns=[f"Region {i+1}" for i in range(len(inference_params["model_a"]))]
)

plot_posterior_samples_model_parameters(df, simulation_params["model_a"])

In [ ]:
from scipy.stats import spearmanr

posterior_model_a = posterior_samples[:, :, :len(inference_params["model_a"])].reshape(-1, len(inference_params["model_a"])).mean(axis=0)
gt_model_a = np.array([v for v in simulation_params["model_a"]])

spearman_correlation = spearmanr(gt_model_a, posterior_model_a)

print(spearman_correlation.correlation, spearman_correlation.pvalue)

In [ ]:
fig_spearman = plt.figure(figsize=(8, 5))
plt.plot(posterior_model_a, gt_model_a, linewidth=0, marker="*", markersize=12, color="blue")
plt.tick_params(axis="both", labelsize=16)

In [ ]:
# data = {
#     f"model_a_R{i+1}": posterior_samples[:, :, :len(inference_params["model_a"])].reshape(-1, len(inference_params["model_a"]))[:, i] for i in range(len(inference_params["model_a"]))
# }
# data["coupling_a"] = posterior_samples[:, :, -2].flatten()
# df = pd.DataFrame(data)
#
# # f2 = plt.figure(figsize=(15, 10))
# with sns.plotting_context(rc={"axes.labelsize":20}):
#     ax2 = sns.pairplot(data=df, kind="hist", height=5)  # , y_vars=["coupling_a"], x_vars=[k for k, _ in data.items() if "model_a" in k])
# ax2.tick_params(axis="both", labelsize=20)
# plt.show()

In [ ]:
data={
    "coupling_a": posterior_samples[:, :, -4].flatten(),
    "nsig": posterior_samples[:, :, -3].flatten(),
    "amplitude": posterior_samples[:, :, -2].flatten(),
    "offset": posterior_samples[:, :, -1].flatten(),
    # "noise": posterior_samples[:, :, -1].flatten(),
}
df = pd.DataFrame(data)

plot_posterior_samples_global_parameters(df, simulation_params)

In [ ]:
#true_mean = np.array(
#    [v for v in simulation_params["model_a"]] + [simulation_params["coupling_a"], simulation_params["nsig"]])
#posterior_mean = posterior_samples.reshape((-1, *posterior_samples.shape[2:])).mean(axis=0)
#posterior_std = posterior_samples.reshape((-1, *posterior_samples.shape[2:])).std(axis=0)
#prior_std = prior_samples.std(axis=0)

In [ ]:
posterior_mean = posterior_samples.mean(axis=1)
posterior_std = posterior_samples.std(axis=1)
prior_std = np.tile(np.array(
    [0.75 for v in simulation_params["model_a"]] + [
        simulation_params["coupling_a"] / 2., simulation_params["nsig"] / 2., 0.5, 0.5]), (len(test_samples), 1))

In [ ]:
np.stack([posterior_mean.mean(axis=0), np.hstack([val for _, val in simulation_params.items()])], axis=1)

In [ ]:
def zscore_gt(true_mean, post_mean):
    return np.abs((post_mean - true_mean) / true_mean)

def zscore_prior(true_mean, post_mean, prior_std):
    return np.abs((post_mean - true_mean) / prior_std)

In [ ]:
zscores = zscore(test_samples, posterior_mean, posterior_std)
shrinkages = shrinkage(prior_std, posterior_std)

zscores_gt = zscore_gt(test_samples, posterior_mean)
zscores_prior = zscore_prior(test_samples, posterior_mean, prior_std)

In [ ]:
f4 = plt.figure(figsize=(12,8))
for i in range(len(simulation_params["model_a"])):
    plt.plot(shrinkages[:, i], zscores[:, i], color=(0, i / 10.0, 1, 1), linewidth=0, marker=".", markersize=8, alpha=0.01)
    plt.plot(shrinkages.mean(axis=0)[i], zscores.mean(axis=0)[i], color=(0, i / 10.0, 1, 1), linewidth=0, marker="*", markersize=12, label=f"model_a_R{i+1}")
plt.plot(shrinkages[:, -4], zscores[:, -4], color="red", linewidth=0, marker=".", markersize=8, alpha=0.01)
plt.plot(shrinkages.mean(axis=0)[-4], zscores.mean(axis=0)[-4], color="red", linewidth=0, marker="*", markersize=12, label="coupling_a")
# plt.plot(shrinkages[:, -3], zscores[:, -3], color="green", linewidth=0, marker=".", markersize=8, alpha=0.1)
# plt.plot(shrinkages.mean(axis=0)[-3], zscores.mean(axis=0)[-3], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.legend(fontsize=16)
plt.tick_params(axis="both", labelsize=16)
plt.plot();

In [ ]:
f5 = plt.figure(figsize=(12,8))
for i in range(len(simulation_params["model_a"])):
    plt.plot(shrinkages[:, i], zscores_gt[:, i], color=(0, i / 10.0, 1, 1), linewidth=0, marker=".", markersize=8, alpha=0.1)
    plt.plot(shrinkages.mean(axis=0)[i], zscores_gt.mean(axis=0)[i], color=(0, i / 10.0, 1, 1), linewidth=0, marker="*", markersize=12, label=f"model_a_R{i+1}")
plt.plot(shrinkages[:, -4], zscores_gt[:, -4], color="red", linewidth=0, marker=".", markersize=8, alpha=0.1)
plt.plot(shrinkages.mean(axis=0)[-4], zscores_gt.mean(axis=0)[-4], color="red", linewidth=0, marker="*", markersize=12, label="coupling_a")
# plt.plot(shrinkages[:, -3], zscores[:, -3], color="green", linewidth=0, marker=".", markersize=8, alpha=0.1)
# plt.plot(shrinkages.mean(axis=0)[-3], zscores.mean(axis=0)[-3], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.legend(fontsize=16)
plt.ylim(-1, 10)
plt.tick_params(axis="both", labelsize=16)
plt.plot();

In [ ]:
f6 = plt.figure(figsize=(12,8))
for i in range(len(simulation_params["model_a"])):
    plt.plot(shrinkages[:, i], zscores_prior[:, i], color=(0, i / 10.0, 1, 1), linewidth=0, marker=".", markersize=8, alpha=0.1)
    plt.plot(shrinkages.mean(axis=0)[i], zscores_prior.mean(axis=0)[i], color=(0, i / 10.0, 1, 1), linewidth=0, marker="*", markersize=12, label=f"model_a_R{i+1}")
plt.plot(shrinkages[:, -4], zscores_prior[:, -4], color="red", linewidth=0, marker=".", markersize=8, alpha=0.1)
plt.plot(shrinkages.mean(axis=0)[-4], zscores_prior.mean(axis=0)[-4], color="red", linewidth=0, marker="*", markersize=12, label="coupling_a")
# plt.plot(shrinkages[:, -3], zscores[:, -3], color="green", linewidth=0, marker=".", markersize=8, alpha=0.1)
# plt.plot(shrinkages.mean(axis=0)[-3], zscores.mean(axis=0)[-3], color="green", linewidth=0, marker="*", markersize=12, label="nsig")
plt.legend(fontsize=16)
plt.ylim(-1, 10)
plt.tick_params(axis="both", labelsize=16)
plt.plot();

In [ ]:
delete = False
if delete:
    os.remove(f"sbi_data/posterior_samples_{run_id}.npy")
    os.remove(f"sbi_data/prior_samples_{run_id}.npy")
    os.remove(f"sbi_data/simulation_{run_id}.npy")
    os.remove(f"sbi_data/training_sims_{run_id}.npy")
    os.remove(f"sbi_data/test_sims_{run_id}.npy")
    os.remove(f"sbi_data/test_samples_{run_id}.npy")
    os.remove(f"sbi_data/sim_params_{run_id}.json")
    os.remove(f"sbi_data/inference_params_{run_id}.json")
    os.remove(f"sbi_data/summary_{run_id}.json")